In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split

# 원본 데이터 디렉터리 및 새로운 디렉터리 경로 설정
original_data_dir = "D:\ko_food\한국_음식"
output_dir = "D:\ko_food\한국_음식"

train_dir = os.path.join(output_dir, "train")
val_dir = os.path.join(output_dir, "val")

# train/val 디렉터리 생성
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# 데이터 분리 비율 설정
val_split = 0.2  # 20% 데이터를 검증 데이터로 분리

# 각 대분류(category)의 하위 음식별 데이터를 처리
for category in os.listdir(original_data_dir):
    category_path = os.path.join(original_data_dir, category)
    if not os.path.isdir(category_path):
        continue
    
    for food in os.listdir(category_path):
        food_path = os.path.join(category_path, food)
        if not os.path.isdir(food_path):
            continue
        
        # 이미지 파일 수집
        images = [os.path.join(food_path, img) for img in os.listdir(food_path) if img.endswith((".jpg", ".png", ".jpeg"))]
        
        # train/val로 분리
        train_images, val_images = train_test_split(images, test_size=val_split, random_state=42)
        
        # 각 데이터셋으로 복사
        for img_path in train_images:
            dest_dir = os.path.join(train_dir, category, food)
            os.makedirs(dest_dir, exist_ok=True)
            shutil.copy(img_path, dest_dir)
        
        for img_path in val_images:
            dest_dir = os.path.join(val_dir, category, food)
            os.makedirs(dest_dir, exist_ok=True)
            shutil.copy(img_path, dest_dir)

print("데이터가 'train'과 'val'로 성공적으로 분리되었습니다!")


데이터가 'train'과 'val'로 성공적으로 분리되었습니다!


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models, optimizers



# 하이퍼파라미터 설정
batch_size = 32
image_size = (224, 224)
num_classes = 150
learning_rate = 0.001
epochs = 25

# 데이터 증강 및 변환 설정
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

val_test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# 데이터 로드
train_generator = train_datagen.flow_from_directory(
    'D:/ko_food/reduced_음식/train_processed',  # train 경로
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    'D:/ko_food/reduced_음식/val_processed',  # val 경로
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    'D:/ko_food/reduced_음식/test_processed',  # test 경로
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Pre-trained 모델 로드 및 fine-tuning 설정
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # 사전 학습된 레이어 고정

# 모델 구성
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')  # 150 클래스로 설정
])

# 모델 컴파일
model.compile(
    optimizer=optimizers.Adam(learning_rate=learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 모델 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    verbose=1
)

# 모델 저장
model.save('foodnet_finetuned_model.h5')

# 사전 학습된 레이어 미세 조정 (fine-tuning)
base_model.trainable = True
model.compile(
    optimizer=optimizers.Adam(learning_rate=learning_rate * 0.1),  # 학습률 낮춤
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tuning 학습
history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=25,  # 추가 학습 에포크
    verbose=1
)

# 테스트 데이터셋 평가
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print(f"Test Accuracy: {test_acc * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

c:\Users\human\anaconda3\envs\human-dl-env2\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\human\anaconda3\envs\human-dl-env2\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10
----------


In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# 하이퍼파라미터 설정
batch_size = 32
image_size = (224, 224)
num_classes = 150
learning_rate = 0.001
epochs_initial = 25  # 초기 학습 에포크
epochs_finetune = 10  # fine-tuning 에포크

# 데이터 증강 및 변환 설정 (train 데이터셋에서 val 및 test 데이터셋 분리)
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.2,
    horizontal_flip=True,
    zoom_range=0.3,
    validation_split=0.2  # 전체 데이터의 20%를 val/test로 분리
)

# 데이터 로드
data_path = 'D:/ko_food/한국_음식/val_processed'

# Train 데이터 생성기 (80% 사용)
train_generator = datagen.flow_from_directory(
    data_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',  # 80% 훈련 데이터
    shuffle=True
)

# Validation 데이터 생성기 (10% 사용)
val_generator = datagen.flow_from_directory(
    data_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',  # 20% 중 10%는 val
    shuffle=True
)

# Test 데이터 생성기 (나머지 10% 사용, shuffle=False)
test_generator = datagen.flow_from_directory(
    data_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',  # 동일한 val 데이터를 사용
    shuffle=False  # 평가 시 데이터 순서 고정
)

# Pre-trained 모델 로드 및 초기 설정
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # 사전 학습된 레이어 고정

# 모델 구성
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')  # 150 클래스로 설정
])

# 모델 컴파일
model.compile(
    optimizer=optimizers.Adam(learning_rate=learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# EarlyStopping 및 ReduceLROnPlateau 설정
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6
)

# 초기 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs_initial,
    callbacks=[early_stopping, lr_scheduler],
    verbose=1
)

# 모델 저장
model.save('foodnet_finetuned_model_initial.h5')

# 일부 레이어 열기
for layer in base_model.layers[:-20]:  # 상위 20개 레이어 제외하고 고정
    layer.trainable = False
for layer in base_model.layers[-20:]:  # 상위 20개 레이어만 학습 가능
    layer.trainable = True

# 모델 재컴파일
model.compile(
    optimizer=optimizers.Adam(learning_rate=learning_rate * 0.1),  # 낮은 학습률로 재설정
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tuning 학습
history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs_finetune,
    callbacks=[early_stopping, lr_scheduler],
    verbose=1
)

# Fine-tuning 후 모델 저장
model.save('foodnet_finetuned_model_final.h5')

# 테스트 데이터셋 평가
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print(f"Test Accuracy: {test_acc * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")



Found 23811 images belonging to 150 classes.
Found 5889 images belonging to 150 classes.
Found 5889 images belonging to 150 classes.


Epoch 1/25


745/745 [==============================] - 968s 1s/step - loss: 5.0193 - accuracy: 0.0050 - val_loss: 5.0105 - val_accuracy: 0.0073 - lr: 0.0010
Epoch 2/25
745/745 [==============================] - 748s 1s/step - loss: 5.0118 - accuracy: 0.0068 - val_loss: 5.0212 - val_accuracy: 0.0073 - lr: 0.0010
Epoch 3/25
745/745 [==============================] - 763s 1s/step - loss: 5.0117 - accuracy: 0.0070 - val_loss: 5.0104 - val_accuracy: 0.0073 - lr: 0.0010
Epoch 4/25
745/745 [==============================] - 782s 1s/step - loss: 5.0116 - accuracy: 0.0060 - val_loss: 5.0104 - val_accuracy: 0.0073 - lr: 0.0010
Epoch 5/25
745/745 [==============================] - 807s 1s/step - loss: 5.0115 - accuracy: 0.0070 - val_loss: 5.0104 - val_accuracy: 0.0073 - lr: 0.0010
Epoch 6/25
745/745 [==============================] - 871s 1s/step - loss: 5.0115 

c:\Users\human\anaconda3\envs\human-dl-env2\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/10
745/745 [==============================] - 694s 923ms/step - loss: 5.1040 - accuracy: 0.0071 - val_loss: 5.0104 - val_accuracy: 0.0073 - lr: 1.0000e-04
Epoch 2/10
745/745 [==============================] - 685s 919ms/step - loss: 5.0486 - accuracy: 0.0079 - val_loss: 5.0104 - val_accuracy: 0.0073 - lr: 1.0000e-04
Epoch 3/10
745/745 [==============================] - 685s 919ms/step - loss: 5.0364 - accuracy: 0.0065 - val_loss: 5.0104 - val_accuracy: 0.0073 - lr: 1.0000e-04
Epoch 4/10
745/745 [==============================] - 686s 921ms/step - loss: 5.0256 - accuracy: 0.0066 - val_loss: 5.0104 - val_accuracy: 0.0073 - lr: 1.0000e-04
Epoch 5/10
745/745 [==============================] - 687s 922ms/step - loss: 5.0224 - accuracy: 0.0076 - val_loss: 5.0104 - val_accuracy: 0.0073 - lr: 5.0000e-05
Epoch 6/10
185/185 [==============================] - 137s 740ms/step - loss: 5.0104 - accuracy: 0.0073
Test Accuracy: 0.73%
Test Loss: 5.0104
